In [1]:
import signac
import numpy as np
import matplotlib.pyplot as plt
import freud
import gsd
import hoomd
import gsd.pygsd
import gsd.hoomd

## RDF for PTB7

In [18]:
data_path = "/home/erjank_project/ptb7-project/workspace"

project = signac.get_project(data_path)

In [17]:
state_dict = {"size":"medium",
              "molecule": "PTB7_5mer_smiles",
              "process":"quench",
              "density": 0.9,
              "kT_reduced": 1.7}

job_list = project.find_jobs(state_dict)

In [4]:
def checkkey(state_dict, key): 
      
    if state_dict[key] ==  "PTB7_10mer_smiles":
        print("10mer")
        mol = "10mer"
    elif state_dict[key] ==  "PTB7_5mer_smiles":
        print("5mer")
        mol = "5mer"
        return mol

key = 'molecule'        
        
checkkey(state_dict, key)         
        
        
    #if key in state_dict.keys(): 
     #   print(state_dict[key])
        
        #mol = state_dict[key]
    #else: 
    #    print("Not present") 
#checkKey(state_dict, key) 

5mer


'5mer'

In [14]:
for job in job_list:
    rdf_path = job.fn("trajectory.gsd")
    print(job)

f082f27bfe6890e697a969d934b678f0
fe36ee4052a01cae041ef5988477de10
65feb942cb0b50717126f2962839c708
639f334452ca975a425617708f6b2c39
32a6f4ec357646c780654e1528dff84b
ae3549e69df51ff0ddcaa887f1be44f6
8772b23514d891a8ccccf3c9285f8544
8a5d42d03f1bfd6d1e0fa0e5495e8837
c416c7bebdae90a23eee7b808b10b1ed
f66ea0b83e09c1f3637ab58150272f22
af95362e840c5599fcba670a0daf4975


In [6]:
print(mol)

NameError: name 'mol' is not defined

In [ ]:
def atom_type_pos(frame, atom_types): 
    positions = []
    for idx, type_id in enumerate(frame.particles.typeid):
        if frame.particles.types[type_id] in atom_types:
            positions.append(frame.particles.position[idx])
    return positions

In [ ]:
def rdf_from_gsd(rdf_path,
                 atom_types='all',
                 r_max = None,
                 r_min = 0.1,
                 nbins = 50,
                 start = 100):
    
    
    f = gsd.pygsd.GSDFile(open(rdf_path, "rb"))
    trajectory = gsd.hoomd.HOOMDTrajectory(f) 
    if r_max is None:
        r_max = max(trajectory[-1].configuration.box[:3]) * 0.45 
    freud_rdf = freud.density.RDF(bins=nbins, r_max=r_max, r_min=r_min)
    for frame in trajectory[start:]:
        if atom_types == 'all':
            freud_rdf.compute(system=frame, reset=False)
        else:
            query_points = atom_type_pos(frame, atom_types)
            box = frame.configuration.box
            freud_rdf.compute(system=(box, query_points), reset = False)
        
 
    x = freud_rdf.bin_centers
    y = freud_rdf.rdf
    filename = job.fn('{}-trajectory.txt').format(atom_types[0])
    np.savetxt(filename, np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")
    f.close()
    return freud_rdf, filename

In [ ]:
def rdf_analysis(job_list): 
    
    for job in job_list:
        rdf_path = job.fn("trajectory.gsd")
        freud_rdf, filename = rdf_from_gsd(rdf_path, atom_types=['f'], start = 95)
        txt_path = job.fn('f-trajectory.txt')
        

    line = np.genfromtxt(txt_path, names = True, delimiter = ",") 
    
    a_type = (txt_path[-16])
    print(a_type)

    x = line["bin_centers"]
    y = line["rdf"]
    
    plt.plot(x, y, color = '#ffc08a')

    plt.xlabel("r")
    plt.ylabel("g(r)")
    plt.ylim(0, 1.6)
    plt.title(state_dict)
    
    mypath = "/home/emilyepstein/projects/ptb7-itic/ptb7-project/jup_notebooks"
    
    pngpath = "{}/plots-ptb7/{}-{}.png".format(mypath, a_type, job)
    #pngpath = "{}/plots-ptb7/{}.png".format(mypath, job)
    
    plt.savefig(pngpath)
    
    #still looking for atom type in title

In [ ]:
rdf_analysis(job_list)

for plotting max:

In [ ]:
#ymax = max(y)
#n_max = y.argmax()
#plt.plot(x[n_max],y[n_max],'o')
#n_min = y.argmin() 
#plt.plot(x[n_min],y[n_min],'x')
#
##unsure how many decimals?
#plt.annotate("local max {}".format("%10.5f"% y[n_max]), xy=(x[n_max],y[n_max]), xytext=(x[n_max]+15,y[n_max]+15), textcoords='offset points', arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=-0.2"))
#
#print("max is:" "%10.5f"% y[n_max]) 

## RDF for ITIC

In [ ]:
data_path = "/home/erjank_project/itic-project/workspace"

project = signac.get_project(data_path)

In [ ]:
state_dict={"size": "small", 
            "process": "quench",
            "kT_reduced":1.2, 
            "density":0.9}

job_list = project.find_jobs(state_dict)

#print(project.detect_schema())

In [ ]:
for job in job_list:
    rdf_path = job.fn("trajectory.gsd")
    print(job)

#### note: atom types defined above

In [ ]:
def rdf_from_gsd(rdf_path,
                 atom_types='all',
                 r_max = None,
                 r_min = 0.1,
                 nbins = 50,
                 start = 100):
    
    
    f = gsd.pygsd.GSDFile(open(rdf_path, "rb"))
    trajectory = gsd.hoomd.HOOMDTrajectory(f) 
    if r_max is None:
        r_max = max(trajectory[-1].configuration.box[:3]) * 0.45 
    freud_rdf = freud.density.RDF(bins=nbins, r_max=r_max, r_min=r_min)
    for frame in trajectory[start:]:
        if atom_types == 'all':
            freud_rdf.compute(system=frame, reset=False)
        else:
            query_points = atom_type_pos(frame, atom_types)
            box = frame.configuration.box
            freud_rdf.compute(system=(box, query_points), reset = False)
        
 
    x = freud_rdf.bin_centers
    y = freud_rdf.rdf
    filename = job.fn('{}-trajectory.txt').format(atom_types[0])
    np.savetxt(filename, np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")
    f.close()
    return freud_rdf, filename

In [ ]:
def rdf_analysis(job_list): 
    
    for job in job_list:
        rdf_path = job.fn("trajectory.gsd")
        freud_rdf, filename = rdf_from_gsd(rdf_path, atom_types=['ss'], start = 0)
        txt_path = job.fn('ss-trajectory.txt')

    line = np.genfromtxt(txt_path, names = True, delimiter = ",") 

    x = line["bin_centers"]
    y = line["rdf"]
    
    plt.plot(x, y, color = '#ffc08a')

    plt.xlabel("r")
    plt.ylabel("g(r)")
    plt.ylim(0, 3.0)
    plt.title(state_dict)
    
    ppath = "/home/emilyepstein/projects/ptb7-itic/ptb7-project/jup_notebooks/plots-itic/{}.png".format(job)
    
    plt.savefig(ppath)
    
    #still looking for atom type in title

In [ ]:
rdf_analysis(job_list)